In [6]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
from torchvision import transforms as T
from tqdm import tqdm
import gc

class CFG:
    # General
    num_workers = 4
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Data paths
    data_dir = './'
    labels_csv_path = os.path.join(data_dir, 'labels.csv')
    sample_submission_path = os.path.join(data_dir, 'sample_submission.csv')
    test_img_dir = os.path.join(data_dir, 'test')

    # Model
    model_name = 'tf_efficientnet_b4_ns'
    img_size = 384
    model_paths = [
        'models/tf_efficientnet_b4_ns_fold0_best.pth',
        'models/tf_efficientnet_b4_ns_fold1_best.pth',
        'models/tf_efficientnet_b4_ns_fold2_best.pth'
    ]

    # Inference
    batch_size = 32
    # This threshold was found by optimizing F1 on the OOF predictions
    threshold = 0.9897

# Load label mappings
labels_df = pd.read_csv(CFG.labels_csv_path)
CFG.attr_ids = labels_df['attribute_id'].values
CFG.attr_id_to_idx = {attr_id: i for i, attr_id in enumerate(CFG.attr_ids)}
CFG.idx_to_attr_id = {i: attr_id for i, attr_id in enumerate(CFG.attr_ids)}
CFG.num_classes = len(labels_df)

# Clean up memory
torch.cuda.empty_cache()
gc.collect()

6

In [7]:
def get_test_transforms():
    # This transform matches the one used for OOF generation which proved to be correct.
    # 1. Squishing resize: T.Resize((h, w)) to match training.
    # 2. TF Normalization: mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5] for `tf_` models.
    return T.Compose([
        T.Resize((CFG.img_size, CFG.img_size)),
        T.ToTensor(),
        T.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5]),
    ])

class iMetTestDataset(Dataset):
    def __init__(self, df, transforms=None):
        self.df = df
        self.filepaths = df['filepath'].values
        self.transforms = transforms

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        filepath = self.filepaths[idx]
        image = Image.open(filepath).convert('RGB')
        
        if self.transforms:
            image = self.transforms(image)
            
        return image

class iMetModel(nn.Module):
    def __init__(self, model_name, pretrained=False):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained, num_classes=CFG.num_classes)

    def forward(self, x):
        return self.model(x)

In [8]:
## Prepare Test Data
sub_df = pd.read_csv(CFG.sample_submission_path)
sub_df['filepath'] = sub_df['id'].apply(lambda x: os.path.join(CFG.test_img_dir, x + '.png'))
display(sub_df.head())

test_dataset = iMetTestDataset(sub_df, transforms=get_test_transforms())
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=CFG.num_workers)

,id,attribute_ids,filepath
0,347c119163f84420f10f7a8126c1b8a2,0 1 2,./test/347c119163f84420f10f7a8126c1b8a2.png
1,98c91458324cba5415c5f5d8ead68328,0 1 2,./test/98c91458324cba5415c5f5d8ead68328.png
2,3f75d332f579af62ff88d369c0736c76,0 1 2,./test/3f75d332f579af62ff88d369c0736c76.png
3,3fa35a29218b7449c8f03e2a368a708d,0 1 2,./test/3fa35a29218b7449c8f03e2a368a708d.png
4,c848b91558e4edd8034cb7d334b4e448,0 1 2,./test/c848b91558e4edd8034cb7d334b4e448.png


In [9]:
## Inference (Ensemble)

# Pre-allocate array for summed predictions for memory efficiency
n_samples = len(test_dataset)
summed_preds = np.zeros((n_samples, CFG.num_classes), dtype=np.float32)

for i, model_path in enumerate(CFG.model_paths):
    print(f"--- Inferencing with model {i+1}/{len(CFG.model_paths)}: {model_path} ---")
    
    # Load model
    model = iMetModel(CFG.model_name, pretrained=False).to(CFG.device)
    model.load_state_dict(torch.load(model_path))
    model.eval()

    pbar = tqdm(test_loader, desc=f"Predicting (Model {i+1})")
    current_pos = 0
    with torch.no_grad():
        for images in pbar:
            images = images.to(CFG.device)
            logits = model(images)
            preds = logits.sigmoid().cpu().numpy()
            
            batch_size = images.size(0)
            summed_preds[current_pos : current_pos + batch_size] += preds
            current_pos += batch_size
            
    # Clean up memory after each model
    del model
    torch.cuda.empty_cache()
    gc.collect()

# Average the predictions
all_preds = summed_preds / len(CFG.model_paths)
print("\nEnsemble predictions calculated.")

--- Inferencing with model 1/3: models/tf_efficientnet_b4_ns_fold0_best.pth ---


/app/.pip-target/timm/models/_factory.py:138: UserWarning: Mapping deprecated model name tf_efficientnet_b4_ns to current tf_efficientnet_b4.ns_jft_in1k.
  model = create_fn(


/tmp/ipykernel_6132/2155724467.py:12: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path))


Predicting (Model 1):   0%|          | 0/667 [00:00<?, ?it/s]

Predicting (Model 1):   0%|          | 1/667 [00:00<05:10,  2.15it/s]

...

Predicting (Model 3): 100%|█████████▉| 664/667 [01:33<00:00,  7.13it/s]

Predicting (Model 3): 100%|█████████▉| 665/667 [01:33<00:00,  7.14it/s]

Predicting (Model 3): 100%|█████████▉| 666/667 [01:33<00:00,  7.13it/s]

Predicting (Model 3): 100%|██████████| 667/667 [01:33<00:00,  7.11it/s]


Ensemble predictions calculated.


In [10]:
## Create Submission
predictions = []
for pred_row in tqdm(all_preds, desc="Formatting submission"):
    # Apply threshold
    pred_labels = (pred_row > CFG.threshold).astype(int)
    
    # If no labels are predicted, take the one with the highest probability
    if pred_labels.sum() == 0:
        pred_labels[pred_row.argmax()] = 1
        
    # Convert indices to attribute_ids
    attr_ids = [CFG.idx_to_attr_id[i] for i, label in enumerate(pred_labels) if label == 1]
    predictions.append(' '.join(map(str, attr_ids)))

sub_df['attribute_ids'] = predictions
sub_df[['id', 'attribute_ids']].to_csv('submission.csv', index=False)
print("Submission file created successfully!")
display(sub_df.head())

Formatting submission:   0%|          | 0/21318 [00:00<?, ?it/s]

Formatting submission:   2%|▏         | 458/21318 [00:00<00:04, 4573.28it/s]

Formatting submission:   4%|▍         | 916/21318 [00:00<00:04, 4572.32it/s]

Formatting submission:   6%|▋         | 1381/21318 [00:00<00:04, 4604.36it/s]

Formatting submission:   9%|▊         | 1842/21318 [00:00<00:04, 4572.31it/s]

...

Formatting submission:  98%|█████████▊| 20798/21318 [00:04<00:00, 4649.55it/s]

Formatting submission: 100%|█████████▉| 21266/21318 [00:04<00:00, 4657.01it/s]

Formatting submission: 100%|██████████| 21318/21318 [00:04<00:00, 4597.34it/s]

Submission file created successfully!


,id,attribute_ids,filepath
0,347c119163f84420f10f7a8126c1b8a2,584 1071 2401,./test/347c119163f84420f10f7a8126c1b8a2.png
1,98c91458324cba5415c5f5d8ead68328,2663,./test/98c91458324cba5415c5f5d8ead68328.png
2,3f75d332f579af62ff88d369c0736c76,10,./test/3f75d332f579af62ff88d369c0736c76.png
3,3fa35a29218b7449c8f03e2a368a708d,1591,./test/3fa35a29218b7449c8f03e2a368a708d.png
4,c848b91558e4edd8034cb7d334b4e448,1537,./test/c848b91558e4edd8034cb7d334b4e448.png
